In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import sys, os
from itertools import product
sys.path.append('..')
from collections import defaultdict
import seaborn
import numpy as np
import pandas as pd
from experiments.utils import *
pd.set_option("display.max_rows",999)

In [25]:
experiments_MACCSFP = load_results(fingerprint="PubchemFP")

Coundn't load 5ht7 uncertainty_sampling 20
Coundn't load 5ht7 random_query 20
Coundn't load 5ht7 czarnecki 20
Couldn't load 5ht7 multiple 20 0.1
Couldn't load 5ht7 multiple 20 0.2
Couldn't load 5ht7 multiple 20 0.3
Couldn't load 5ht7 multiple 20 0.4
Couldn't load 5ht7 multiple 20 0.5
Couldn't load 5ht7 multiple 20 0.6
Couldn't load 5ht7 multiple 20 0.7
Couldn't load 5ht7 multiple 20 0.8
Couldn't load 5ht7 multiple 20 0.9
Couldn't load 5ht7 chen_krause 20 50
Couldn't load 5ht7 chen_krause 20 200
Couldn't load 5ht7 chen_krause 20 500
Coundn't load 5ht7 uncertainty_sampling 50
Coundn't load 5ht7 random_query 50
Coundn't load 5ht7 czarnecki 50
Couldn't load 5ht7 multiple 50 0.1
Couldn't load 5ht7 multiple 50 0.2
Couldn't load 5ht7 multiple 50 0.3
Couldn't load 5ht7 multiple 50 0.4
Couldn't load 5ht7 multiple 50 0.5
Couldn't load 5ht7 multiple 50 0.6
Couldn't load 5ht7 multiple 50 0.7
Couldn't load 5ht7 multiple 50 0.8
Couldn't load 5ht7 multiple 50 0.9
Couldn't load 5ht7 chen_krause 50 50


In [26]:
exps = experiments_MACCSFP.values()[0]
len(exps)

42

In [27]:
for e in exps:
    print e.misc['mean_monitor']['wac_score_concept'][-1]

0.802662891599
0.806230030874
0.802662891599
0.802662891599
0.802662891599
0.802662891599
0.802662891599
0.802662891599
0.802662891599
0.802662891599
0.802662891599
0.802662891599
0.802662891599
0.802662891599
0.802662891599
0.802662891599
0.802662891599
0.802662891599
0.802662891599
0.802662891599
0.802662891599
0.802662891599
0.802662891599
0.802662891599
0.802662891599
0.802662891599
0.802662891599
0.802662891599
0.802662891599
0.802662891599
0.802662891599
0.802662891599
0.802662891599
0.802662891599
0.802662891599
0.802662891599
0.802662891599
0.802662891599
0.802662891599
0.802662891599
0.802662891599
0.802662891599


In [22]:
for i in range(5):
    var = []
    for e in exps:
        var.append(e.monitors[i]['wac_score_concept'][-1])
    print np.std(var)

0.00362988902349
0.00448398055843
0.00200599130245
0.00491791416085
0.00390911125606


In [4]:
def load_results(fingerprint="ExtFP", batch_sizes=[20, 50, 100]):

    proteins = ['5ht7','hiv_integrase','h1','cathepsin','M1', '5ht6']
    strategies = ['uncertainty_sampling', 'random_query', 'czarnecki']
    experiments = {p+'_'+str(bs): [] for p in proteins for bs in batch_sizes}
    n_loaded = 0
    n_all = len(proteins) * len(batch_sizes) * (len(strategies) + 4*9 + 3) # gtfo pisałem na szybko
    for p in proteins:
        for batch_size in batch_sizes:
            for strat in strategies:
                try:
                    exp_name = "fit_SVMTAN_%s_%s_%s_%s" % (strat, p, fingerprint, str(batch_size))
                    exp = get_experiment_results(exp_name)
                    experiments[p+'_'+str(batch_size)] += exp.experiments
                    n_loaded += 1
                except:
                    print "Coundn't load", p, strat, batch_size
                    continue
            for c in list(np.linspace(0.1, 0.9, 9)):
                
                exp_name = "fit_SVMTAN_multiple_pick_best_prim_c_%.2f_%s_%s_%s" % (c, p, fingerprint, str(batch_size))
                
                if fingerprint == "ExtFP":
                     exp_name = "fit_SVMTAN_multiple_pick_best_c_%.2f_%s_%s_%s" % (c, p, fingerprint, str(batch_size))
                
                try:
                    exp = get_experiment_results(exp_name)
                    assert len(exp.experiments) == 1
                    experiments[p+'_'+str(batch_size)].append(exp.experiments[0])
                    n_loaded += 1
                except:
                    print "Couldn't load", p, "multiple", batch_size, c
                    continue

                exp_name = "fit_SVMTAN_czarnecki_two_clusters_c_%.2f_%s_%s_%s" % (c, p, fingerprint, str(batch_size))
 
                try:
                    exp = get_experiment_results(exp_name)
                    assert len(exp.experiments) == 1
                    experiments[p+'_'+str(batch_size)].append(exp.experiments[0])
                    n_loaded += 1
                except:
                    print "Couldn't load", p, "multiple", batch_size, c
                    continue
                    
                exp_name = "fit_SVMTAN_czarnecki_two_clusters_sorensen_c_%.2f_%s_%s_%s" % (c, p, fingerprint, str(batch_size))
 
                try:
                    exp = get_experiment_results(exp_name)
                    # Hack to make it distinguishable
                    for experiment in exp.experiments:
                        experiment.config["strategy"] = "czarnecki_two_clusters_sorensen"
                        
                    assert len(exp.experiments) == 1
                    experiments[p+'_'+str(batch_size)].append(exp.experiments[0])
                    n_loaded += 1
                except:
                    print "Couldn't load", p, "czarnecki_two_clusters_sorensen", batch_size, c
                    continue
                    
                    
                exp_name = "fit_SVMTAN_quasi_greedy_batch_c_%.2f_%s_%s_%s" % (c, p, fingerprint, str(batch_size))
                try:
                    exp = get_experiment_results(exp_name)
                    assert len(exp.experiments) == 1
                    experiments[p+'_'+str(batch_size)].append(exp.experiments[0])
                    n_loaded += 1
                except:
                    print "Couldn't load", p, "quasi_greedy", batch_size, c
                    continue

            for h in [50,200,500]:
                exp_name = "fit_SVMTAN_multiple_pick_best_h_%d_%s_%s_%s" % (h, p, fingerprint, str(batch_size))
                try:
                    exp = get_experiment_results(exp_name)
                    assert len(exp.experiments) == 1
                    experiments[p+'_'+str(batch_size)].append(exp.experiments[0])
                    n_loaded += 1
                except:
                    print "Couldn't load", p, "chen_krause", batch_size, h
                    continue
    
    print "Loaded %d/%d" % (n_loaded, n_all)
    return experiments

def count_strat_experiments(experiments):
    strats = []

    for key, protein_experiments in experiments.iteritems():
        for e in protein_experiments:
            strats.append(e.config['strategy'])

    return Counter(strats)

def get_best_per_strategy(experiments, metric='auc_wac_score_concept'):

    strategies = ['chen_krause', 'uncertainty_sampling', 'random_query', 'czarnecki', 'quasi_greedy_batch',
                  'multiple_pick_best', 'czarnecki_two_clusters', 'czarnecki_two_clusters_sorensen']
    best_protein_exp = {k: {s: [None, 0] for s in strategies} for k in experiments.keys()}
    for key, protein_experiments in experiments.iteritems():
        for e in protein_experiments:
            strat = e.config['strategy']
            if e.results[metric] >= best_protein_exp[key][strat][1]:
                best_protein_exp[key][strat][0] = e
                best_protein_exp[key][strat][1] = e.results[metric]

    return {k: {s: e[0] for s, e in exps.iteritems()} for k, exps in best_protein_exp.iteritems()}


def plot_monitors(experiments, keys='metrics', folds='mean', figsize=(15,15), 
                  filter_out=['czarnecki', 'quasi_greedy_batch']):
    import matplotlib.pylab as plt

    assert folds in ['all', 'mean']
    assert keys in ['metrics', 'times']

    if keys == 'times':
        include = ['unlabeled_test_times',
                   'grid_times',
                   'strat_times',
                   'concept_test_times']
    elif keys == "metrics":
        include = []
        for metr in ["wac_score"]:
            for dataset in ['concept', 'cluster_B_concept']:
#                             ["concept", "unlabeled", "cluster_A_concept", "cluster_B_concept",
#                             "cluster_A_unlabeled", "cluster_B_unlabeled"]:
                include.append(metr+"_"+dataset)


    keys = [k for k in experiments.values()[0].monitors[0].keys() if k in include]

    f, axes = plt.subplots(len(keys), 1)
    f.set_figheight(figsize[1])
    f.set_figwidth(figsize[0])

    assert folds == 'mean'

    if folds == 'mean':
        for ax, key in zip(axes, keys):
            for strat, e in experiments.iteritems():
                if e is None or strat in filter_out:
                    continue
                pd.DataFrame({strat: e.misc['mean_monitor'][key]}).plot(title=key, ax=ax)
                ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
    else:
        for ax, key in zip(axes, keys):
            for e in experiments:
                for mon in e.monitors:
                    pd.DataFrame({e.name: mon[key]}).plot(title=key, ax=ax)
                    ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

In [5]:
def count_wins(experiments, metric = 'auc_wac_score_concept', filter_out=[]):

    strategies = ['chen_krause', 
                  'uncertainty_sampling', 
                  'quasi_greedy_batch', 
                  'random_query', 
                  'multiple_pick_best',
                  'czarnecki', 
                  'czarnecki_two_clusters', 
                  'czarnecki_two_clusters_sorensen']

    wins = {strat: 0 for strat in strategies if strat not in filter_out}
    diffs = defaultdict(float)
    best_exps = {p: None for p in experiments.keys()}

    for p, protein_exps in experiments.iteritems():
        
        if len(protein_exps) != 0:
            best_score = (None, -np.inf)

            for e in protein_exps:
                if e.config['strategy'] in filter_out:
                    continue
                    
                if metric[:5] == 'fast_':
                    metric = metric[5:]
                    metric_scores = e.misc['mean_monitor'][metric]
                    high_score = max(metric_scores)
                    ind = np.where(metric_scores >= 0.9 * high_score)[0][0]
                    exp_score = e.monitors[0]['iter'] - ind
                elif 'auc' not in metric and 'mean' not in metric:
                    exp_score = e.misc['mean_monitor'][metric][-1]
                else:
                    exp_score = e.results[metric]
                    
                if exp_score >= best_score[1]:
                    best_score = (e.config['strategy'], exp_score)
                    best_exps[p] = e

            if best_score[0] <= 0:
                print best_score
            wins[best_score[0]] += 1
            if 'auc' not in metric and 'mean' not in metric:
                diffs[best_score[0]] += best_score[1] - max([e.misc['mean_monitor'][metric][-1] for e in protein_exps
                                                             if e.config['strategy'] != best_score[0]])
            else:
                diffs[best_score[0]] += best_score[1] - max([e.results[metric] for e in protein_exps
                                                             if e.config['strategy'] != best_score[0]])
        else:
            print "Zero length for "+p

            
    for k in wins:
        diffs[k] /= max(1.0, float(wins[k]))
         
            
    keys = wins.keys()
    keys_proteins = best_exps.keys()
    
    wins_df = pd.DataFrame([(wins[key], diffs[key]) for key in wins.keys()], index=keys)
    wins_df.columns = ['score', 'avg_diff']

    which_df = pd.DataFrame([(e.config['strategy']) for p, e in best_exps.iteritems()], index=keys_proteins)
    which_df.columns = ["best strategy"]

    return wins_df


In [6]:
def pairwise_count_wins(experiments, metric='auc_wac_score_concept', compare=None):

    strategies = ['chen_krause', 'uncertainty_sampling', 'quasi_greedy_batch', 'random_query', 'multiple_pick_best',
              'czarnecki', 'czarnecki_two_clusters', 'czarnecki_two_clusters_sorensen']

    scores = defaultdict(float)
    wins = defaultdict(float)
    diffs = defaultdict(float)
    
    if compare is None:
        compare = strategies

    for p, protein_exps in experiments.iteritems():
        # Get scores for all strategies
        strategy_scores = defaultdict(float)
        for e in protein_exps:
            
            if e.config["strategy"] in compare:
            
                if metric[:5] == 'fast_':
                    metric = metric[5:]
                    metric_scores = e.misc['mean_monitor'][metric]
                    high_score = max(metric_scores)
                    ind = np.where(metric_scores >= 0.9 * high_score)[0][0]
                    exp_score = e.monitors[0]['iter'] - ind
                elif 'auc' not in metric and 'mean' not in metric:
                    exp_score = e.misc['mean_monitor'][metric][-1]
                else:
                    exp_score = e.results[metric]

                if e.config["strategy"] in ['quasi_greedy_batch', 'multiple_pick_best']:
                    strat = e.config["strategy"] + "_" + str(e.config['strategy_kwargs']['c'])
                elif e.config["strategy"] in ['czarnecki_two_clusters']:
                    strat = "czarnecki_tc_" + str(e.config['strategy_kwargs']['c'])
                elif e.config["strategy"] in ['czarnecki_two_clusters_sorensen']:
                    strat = "czarnecki_tc_sor_" + str(e.config['strategy_kwargs']['c'])
                elif e.config["strategy"] == 'chen_krause':
                    strat = e.config["strategy"] + "_" + str(e.config['strategy_projection_h'])
                else:
                    strat = e.config["strategy"]

                strategy_scores[strat] = max(strategy_scores[strat], exp_score)
        
        for strat1, strat2 in product(strategy_scores, strategy_scores):
            # We don't compare same "types" of strtegies
            if strat1!=strat2 and (strat1[:5] != strat2[:5] or strat1[:9] == "czarnecki"):
                if strategy_scores[strat1] > strategy_scores[strat2]:
                    wins[strat1 + " vs " + strat2] += 1 
                    scores[strat1 + " vs " + strat2] += 1
                    diffs[strat1 + " vs " + strat2] += strategy_scores[strat1] - strategy_scores[strat2]
                else:
                    scores[strat1 + " vs " + strat2] -= 1
                    #diffs[strat1 + " vs " + strat2] += strategy_scores[strat1] - strategy_scores[strat2]
        
        for k in wins:
            diffs[k] /= max(1.0, abs(float(wins[k])))
        
    keys = scores.keys()
    df = pd.DataFrame([(scores[key], diffs[key]) for key in keys], index=keys)
    df.columns = ['score', 'avg_diff']


    return df

In [7]:
from collections import OrderedDict

def average_ranking(experiments, metric="auc_wac_score_concept", filter_out=[]):
    
    strategies = ['chen_krause', 'uncertainty_sampling', 'quasi_greedy_batch', 'random_query', 
                  'multiple_pick_best', 'czarnecki', 'czarnecki_two_clusters', 'czarnecki_two_clusters_sorensen']
    
    rank = defaultdict(list)
    
    for p, protein_exps in experiments.iteritems():

        if len(protein_exps) != 0:
            scores = defaultdict(list)
            for e in protein_exps:

                if e.config['strategy'] in filter_out:
                    continue
                  
                if metric[:5] == 'fast_':
                    metric = metric[5:]
                    metric_scores = e.misc['mean_monitor'][metric]
                    high_score = max(metric_scores)
                    ind = np.where(metric_scores >= 0.9 * high_score)[0][0]
                    exp_score = e.monitors[0]['iter'] - ind
                elif metric[:12] == 'partial_mean':
                    print "test"
                    metric = metric[13:]
                    exp_score = np.mean(e.misc['mean_monitor'][metric][-15:-5])
                elif 'auc' not in metric and 'mean' not in metric:
                    exp_score = e.misc['mean_monitor'][metric][-1]
                else:
                    exp_score = e.results[metric]
                    
                scores[e.config['strategy']].append(exp_score)

            for k, s in scores.iteritems():
                scores[k] = np.max(s)

            # it's a kind of magic
            for i, strat in enumerate(OrderedDict(sorted(scores.items(), key=lambda t: t[1], reverse=True)).keys()): 
                rank[strat].append(i+1)

        else:
            print "Zero length for " + p

    for strat, ranks in rank.iteritems():
        rank[strat] = np.mean(ranks, dtype=float)
    
    strats = rank.keys()
    df = pd.DataFrame([rank[key] for key in strats], index=strats)
    df.columns = ['average_rank']
    return df

In [8]:
def join_experiments(experiment_dicts):
    exp_keys = experiment_dicts[0].keys()
    for e in experiment_dicts:
        assert e.keys() == exp_keys

    joined_experiments = defaultdict(list)
    for e in experiment_dicts:
        for k in exp_keys:
            joined_experiments[k] += e[k]
            
    return joined_experiments